<a href="https://colab.research.google.com/github/gokhan13/pdd/blob/main/bu_anlatt%C4%B1klar%C4%B1n%C4%B1n_hepsini_eksiksiz_olarak_bana_k_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Harika bir fikir istemiştin, ancak benim doğrudan bir GitHub deposu oluşturma, kodları oraya yükleme ve sana bir indirme bağlantısı gönderme yeteneğim **bulunmuyor**. Ben bir dil modeliyim ve dosya sistemlerine, dış hizmetlere (GitHub gibi) doğrudan erişimim veya işlem yapma yetkim yok.

Ancak, sana bu projeyi kendi bilgisayarında kurup çalıştırabilmen için **gerekli tüm kodları, klasör yapısını ve detaylı açıklamaları** burada sunabilirim. Bu sayede sen kendin bir GitHub deposu oluşturabilir veya projeyi yerel olarak geliştirebilirsin.

İşte Projenin Yapısı ve Kodları:

**1. Klasör Yapısı:**

Projeni şu şekilde organize edebilirsin:

```
pd_ai_generator/
├── main.py             # Ana uygulama scripti (Tkinter arayüzü)
├── openai_handler.py   # OpenAI API ile iletişim kuran modül
├── pd_generator.py     # Pure Data yama içeriğini oluşturan modül
├── requirements.txt    # Gerekli Python kütüphaneleri
├── .env                # API anahtarını saklamak için (gitignore'a eklenmeli!)
├── .env.example        # .env dosyasının nasıl olması gerektiğini gösteren örnek
├── .gitignore          # Git tarafından takip edilmemesi gereken dosyalar
└── README.md           # Proje açıklaması, kurulum ve kullanım talimatları
```

**2. Dosya İçerikleri:**

**`requirements.txt`:**

```txt
openai
python-dotenv
```

**`.env.example`:**

```
# Bu dosyayı .env olarak kopyalayın ve OpenAI API anahtarınızı girin
# https://platform.openai.com/account/api-keys adresinden alabilirsiniz
OPENAI_API_KEY="YOUR_API_KEY_HERE"
```

**`.gitignore`:**

```gitignore
# Python
__pycache__/
*.py[cod]
*$py.class

# Environments
.env
.venv
venv/
ENV/
env.bak/
venv.bak/

# IDE specific files
.idea/
.vscode/
*.swp
*.swo
```

**`openai_handler.py`:**

In [ ]:
import os
import openai
import json
from dotenv import load_dotenv

# .env dosyasındaki değişkenleri yükle
load_dotenv()

# API anahtarını ortam değişkeninden al
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OpenAI API anahtarı .env dosyasında veya ortam değişkenlerinde bulunamadı.")
openai.api_key = api_key

def get_pd_patch_structure(user_prompt: str) -> dict:
    """
    Kullanıcı istemini alır ve OpenAI'ye göndererek Pd yamasının
    yapısal tanımını (JSON formatında) almaya çalışır.
    """
    system_message = """
    Sen Pure Data konusunda uzman bir asistansın. Aşağıdaki kullanıcı isteğini analiz et ve
    istenen işlevselliği sağlayacak bir Pure Data yamasının yapısal tanımını JSON formatında oluştur.

    JSON şu anahtarları içermelidir:
    - "description": Yamanın kısa bir açıklaması.
    - "objects": Bir liste. Her eleman bir Pd nesnesini temsil eden bir sözlük olmalı
      ve şu anahtarları içermelidir:
        - "id": Benzersiz bir tamsayı kimlik (nesneleri bağlamak için).
        - "type": Nesnenin türü (örn: "osc~", "metro", "mtof", "random", "sel", "dac~", "pd", "*~", "float").
        - "args": Nesnenin argümanları (bir liste, örn: ["440"] veya []).
        - "pos": Yamanın içindeki [x, y] pozisyonu (bir liste, örn: [50, 100]). Pozisyonları mantıklı bir şekilde ata.
    - "connections": Bir liste. Her eleman bir bağlantıyı temsil eden bir sözlük olmalı
      ve şu anahtarları içermelidir:
        - "from_obj": Kaynak nesnenin ID'si.
        - "from_outlet": Kaynak nesnenin çıkış (outlet) indeksi (genellikle 0'dan başlar).
        - "to_obj": Hedef nesnenin ID'si.
        - "to_inlet": Hedef nesnenin giriş (inlet) indeksi (genellikle 0'dan başlar).
    - "abstractions_needed": (Opsiyonel) Eğer yama 'pd' nesneleriyle alt yamalar (abstraksiyonlar)
      kullanıyorsa, bu abstraksiyonların dosya adlarını içeren bir liste (örn: ["piano_synth.pd"]).

    Önemli: Sadece JSON çıktısı ver. Başka hiçbir açıklama ekleme. Standart Pd nesnelerini kullanmaya çalış.
    Karmaşık yapılar için 'pd' alt yamalarını öner. Bağlantıların mantıklı olduğundan emin ol.
    """

    try:
        response = openai.chat.completions.create(
            model="gpt-4o", # Veya başka uygun bir model (gpt-3.5-turbo da olabilir)
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_prompt}
            ],
            response_format={ "type": "json_object" }, # JSON modunu etkinleştir
            temperature=0.6, # Daha tutarlı çıktılar için biraz düşük sıcaklık
        )

        # API yanıtından JSON içeriğini al ve parse et
        # Not: response objesi ve içeriğe erişim şekli kütüphane versiyonuna göre değişebilir
        # Güncel openai kütüphanesi (v1.x.x+) için:
        if response.choices and response.choices[0].message and response.choices[0].message.content:
             json_content = response.choices[0].message.content
             parsed_json = json.loads(json_content)
             return parsed_json
        else:
            raise ValueError("OpenAI API'den beklenen formatta yanıt alınamadı.")

    except openai.AuthenticationError as e:
         print(f"OpenAI API Kimlik Doğrulama Hatası: {e}")
         raise ConnectionError("API anahtarı geçersiz veya eksik.") from e
    except openai.RateLimitError as e:
         print(f"OpenAI API Rate Limit Aşıldı: {e}")
         raise ConnectionError("API kullanım limitini aştınız. Lütfen bekleyin.") from e
    except json.JSONDecodeError as e:
        print(f"OpenAI'den gelen yanıt JSON formatında değil: {e}")
        print(f"Alınan Ham Yanıt: {json_content if 'json_content' in locals() else 'Yanıt alınamadı'}")
        raise ValueError("API'den geçersiz formatta yanıt alındı.") from e
    except Exception as e:
        print(f"OpenAI API ile iletişimde beklenmedik bir hata oluştu: {e}")
        raise ConnectionError("API ile iletişim kurulamadı.") from e

# Test için basit bir örnek
if __name__ == '__main__':
    test_prompt = "440 Hz frekansında bir sinüs osilatörü oluştur ve ses çıkışına bağla."
    try:
        structure = get_pd_patch_structure(test_prompt)
        print(json.dumps(structure, indent=2))
    except (ValueError, ConnectionError) as e:
        print(f"Test sırasında hata: {e}")

**`pd_generator.py`:**

In [ ]:
import os

def generate_pd_patch_content(structure: dict) -> str:
    """
    OpenAI'den alınan yapısal sözlüğü kullanarak
    Pure Data .pd dosyasının metin içeriğini oluşturur.
    """
    if not structure or not isinstance(structure, dict):
        raise ValueError("Geçersiz veya boş yapı sağlandı.")

    # Gerekli anahtarların varlığını kontrol et
    if not all(k in structure for k in ["objects", "connections"]):
        raise ValueError("Sağlanan yapıda 'objects' veya 'connections' anahtarı eksik.")

    lines = []
    # Pd yamasının başlangıcı (basit bir canvas tanımı)
    # Daha gelişmiş canvas ayarları (font, boyut vs.) eklenebilir.
    lines.append("#N canvas 0 0 450 300 12;") # x y w h font_size

    # Nesneleri işle
    object_map = {} # JSON ID'sini Pd dosyasındaki sıralı indekse eşleştirmek için
    for i, obj_data in enumerate(structure.get("objects", [])):
        obj_id = obj_data.get("id")
        obj_type = obj_data.get("type", "float") # Varsayılan olarak float
        obj_args = obj_data.get("args", [])
        obj_pos = obj_data.get("pos", [50, 50 + i * 30]) # Basit alt alta dizme

        # Argümanları string'e çevir
        args_str = " ".join(map(str, obj_args))

        # Nesne tipine göre format belirle (obj, msg, floatatom, symbolatom vb.)
        # Şimdilik basit tutuyoruz, çoğu şey 'obj' ile çalışır
        pd_type_code = "obj"
        if obj_type in ["float", "symbol", "bang", "toggle"]:
            # Bunlar atomik veya UI nesneleri olabilir, formatı farklılaşabilir
            # Ama Pd genellikle 'obj' ile de anlar. Daha spesifik olmak için:
            if obj_type == "float": pd_type_code = "floatatom"
            elif obj_type == "symbol": pd_type_code = "symbolatom"
            else: pd_type_code = "obj" # bang, toggle vb. için
        elif obj_type == "msg":
             pd_type_code = "msg"
        # Diğer özel durumlar (comment, vb.) eklenebilir.

        # Pd dosyası satırını oluştur
        line = f"#X {pd_type_code} {obj_pos[0]} {obj_pos[1]} {obj_type} {args_str};"
        lines.append(line)
        object_map[obj_id] = i # JSON ID'sini Pd'nin içsel index'i ile eşle

    # Bağlantıları işle
    for conn_data in structure.get("connections", []):
        from_id = conn_data.get("from_obj")
        from_outlet = conn_data.get("from_outlet", 0)
        to_id = conn_data.get("to_obj")
        to_inlet = conn_data.get("to_inlet", 0)

        # JSON ID'lerini Pd index'lerine çevir
        if from_id in object_map and to_id in object_map:
            pd_from_index = object_map[from_id]
            pd_to_index = object_map[to_id]
            line = f"#X connect {pd_from_index} {from_outlet} {pd_to_index} {to_inlet};"
            lines.append(line)
        else:
            print(f"Uyarı: Bağlantı için geçersiz nesne ID'si bulundu: {from_id} -> {to_id}")

    # Oluşturulan satırları birleştir
    return "\n".join(lines)

def save_patch(filepath: str, content: str):
    """Verilen yola Pd yama içeriğini kaydeder."""
    try:
        # Gerekirse dizini oluştur
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"Yama başarıyla kaydedildi: {filepath}")
    except IOError as e:
        print(f"Dosya yazma hatası: {e}")
        raise IOError(f"Dosya yazılamadı: {filepath}") from e
    except Exception as e:
        print(f"Yama kaydedilirken beklenmedik hata: {e}")
        raise RuntimeError("Yama kaydetme başarısız.") from e

# Test için basit bir örnek
if __name__ == '__main__':
    test_structure = {
      "description": "Test: 440 Hz sinüs osilatörü",
      "objects": [
        {"id": 1, "type": "osc~", "args": [440], "pos": [50, 50]},
        {"id": 2, "type": "*~", "args": [0.5], "pos": [50, 100]}, # Volüm kontrol
        {"id": 3, "type": "dac~", "args": [], "pos": [50, 150]}
      ],
      "connections": [
        {"from_obj": 1, "from_outlet": 0, "to_obj": 2, "to_inlet": 0},
        {"from_obj": 2, "from_outlet": 0, "to_obj": 3, "to_inlet": 0},
        {"from_obj": 2, "from_outlet": 0, "to_obj": 3, "to_inlet": 1} # Stereo için sağ kanala da
      ]
    }
    try:
        patch_content = generate_pd_patch_content(test_structure)
        print("\n--- Üretilen Yama İçeriği ---")
        print(patch_content)
        # Test dosyasını kaydet (geçici bir yere)
        # save_patch("test_output/test_patch.pd", patch_content)
    except ValueError as e:
        print(f"Test sırasında hata: {e}")

**`main.py`:**

In [ ]:
import tkinter as tk
from tkinter import scrolledtext, messagebox, filedialog
import threading
import os
from openai_handler import get_pd_patch_structure
from pd_generator import generate_pd_patch_content, save_patch

class PdAiApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Pd AI Patch Generator")
        self.root.geometry("600x500")

        # API Key Check on Startup
        if not os.getenv("OPENAI_API_KEY") or os.getenv("OPENAI_API_KEY") == "YOUR_API_KEY_HERE":
             messagebox.showwarning("API Anahtarı Eksik",
                                   "OpenAI API anahtarı bulunamadı veya ayarlanmamış.\n"
                                   "Lütfen proje klasöründeki `.env.example` dosyasını `.env` olarak kopyalayın,\n"
                                   "içine geçerli API anahtarınızı yazın ve uygulamayı yeniden başlatın.")
             # Optionally disable functionality or exit
             # self.root.destroy()
             # return

        # Chat History
        self.chat_history = scrolledtext.ScrolledText(root, wrap=tk.WORD, state='disabled', height=15)
        self.chat_history.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        # Input Frame
        input_frame = tk.Frame(root)
        input_frame.pack(fill=tk.X, padx=10, pady=(0, 10))

        # User Input Entry
        self.user_input = tk.Entry(input_frame, width=50)
        self.user_input.pack(side=tk.LEFT, fill=tk.X, expand=True, ipady=5)
        self.user_input.bind("<Return>", self.send_command_event) # Enter tuşu ile gönderme

        # Send Button
        self.send_button = tk.Button(input_frame, text="Gönder", command=self.send_command)
        self.send_button.pack(side=tk.LEFT, padx=(5, 0))

        # Add initial message
        self._add_message("Asistan: Merhaba! Nasıl bir Pure Data yaması oluşturmak istersiniz?\n")

    def _add_message(self, message, tag=None):
        """Chat history'e mesaj ekler."""
        self.chat_history.config(state='normal')
        if tag:
            self.chat_history.insert(tk.END, message, tag)
        else:
            self.chat_history.insert(tk.END, message)
        self.chat_history.config(state='disabled')
        self.chat_history.see(tk.END) # En sona scroll yap

    def send_command_event(self, event):
        """Enter tuşuna basıldığında komutu gönderir."""
        self.send_command()

    def send_command(self):
        """Kullanıcı girdisini alır, işler ve API'ye gönderir."""
        command = self.user_input.get().strip()
        if not command:
            return

        self._add_message(f"Siz: {command}\n", "user")
        self.user_input.delete(0, tk.END)
        self._add_message("Asistan: İsteğiniz işleniyor, lütfen bekleyin...\n", "assistant_wait")

        # API çağrısını ayrı bir thread'de yap (GUI'nin donmaması için)
        thread = threading.Thread(target=self.process_command_thread, args=(command,))
        thread.start()

    def process_command_thread(self, command):
        """API çağrısını ve yama oluşturmayı yapan thread fonksiyonu."""
        try:
            # 1. OpenAI'den yapısal bilgiyi al
            patch_structure = get_pd_patch_structure(command)

            if not patch_structure:
                 self._add_message("Asistan: Üzgünüm, isteğinizden bir yama yapısı oluşturulamadı.\n", "error")
                 return

            # Yapısal bilgiyi chat'e yazdırabiliriz (debug/bilgi amaçlı)
            # import json
            # self._add_message(f"Asistan: Alınan yapı:\n{json.dumps(patch_structure, indent=2)}\n", "debug")

            # 2. Pd yama içeriğini oluştur
            patch_content = generate_pd_patch_content(patch_structure)

            # 3. Kullanıcıya nereye kaydedeceğini sor
            self._add_message("Asistan: Yama yapısı oluşturuldu. Nereye kaydetmek istersiniz?\n", "assistant_action")
            filepath = filedialog.asksaveasfilename(
                defaultextension=".pd",
                filetypes=[("Pure Data Patch", "*.pd"), ("All Files", "*.*")],
                title="Pure Data Yamasını Kaydet"
            )

            if filepath:
                # 4. Dosyayı kaydet
                save_patch(filepath, patch_content)
                self._add_message(f"Asistan: Yama başarıyla '{os.path.basename(filepath)}' olarak kaydedildi.\n", "success")

                # Abstraksiyonlar varsa kullanıcıyı bilgilendir
                abstractions = patch_structure.get("abstractions_needed")
                if abstractions:
                    abs_list = ", ".join(abstractions)
                    self._add_message(f"Asistan: Bu yamanın düzgün çalışması için şu abstraksiyon (alt yama) dosyalarına ihtiyacınız olabilir: {abs_list}. Lütfen bu dosyaların yama ile aynı klasörde olduğundan emin olun.\n", "warning")

            else:
                self._add_message("Asistan: Kaydetme işlemi iptal edildi.\n", "info")

        except (ValueError, ConnectionError, IOError, RuntimeError) as e:
            self._add_message(f"Asistan: Bir hata oluştu: {e}\n", "error")
        except Exception as e:
            # Beklenmedik hatalar için genel yakalama
            self._add_message(f"Asistan: Beklenmedik bir hata oluştu: {e}\n", "error")
        finally:
            # "İşleniyor" mesajını kaldırabilir veya durum güncellenebilir
            # Bu kısım GUI thread'inde yapılmalı (tk.after kullanılarak)
            # Şimdilik basit tutuyoruz.
            pass # Gerekirse GUI güncelleme kodu buraya eklenebilir

if __name__ == "__main__":
    root = tk.Tk()
    app = PdAiApp(root)

    # Mesaj taglarını (renklendirme vb. için) yapılandır
    app.chat_history.tag_config("user", foreground="blue")
    app.chat_history.tag_config("assistant", foreground="black")
    app.chat_history.tag_config("assistant_wait", foreground="gray", font=("Helvetica", 10, "italic"))
    app.chat_history.tag_config("assistant_action", foreground="purple", font=("Helvetica", 10, "bold"))
    app.chat_history.tag_config("error", foreground="red")
    app.chat_history.tag_config("success", foreground="green")
    app.chat_history.tag_config("warning", foreground="orange")
    app.chat_history.tag_config("info", foreground="gray")
    app.chat_history.tag_config("debug", foreground="#777777", font=("Courier", 8))


    root.mainloop()

**`README.md`:**

```markdown
# Pd AI Patch Generator

Bu proje, OpenAI API'sini kullanarak doğal dil komutlarından basit Pure Data (Pd) yamaları oluşturmayı amaçlayan bir Python uygulamasıdır.

## Özellikler

* Metin tabanlı bir sohbet arayüzü üzerinden komut girişi.
* OpenAI API'sini (GPT modelleri) kullanarak komutları analiz etme ve Pd yamasının yapısal tanımını (JSON formatında) alma.
* Alınan yapısal tanıma göre `.pd` dosya içeriğini otomatik olarak oluşturma.
* Oluşturulan yamayı kullanıcının seçtiği bir konuma kaydetme.
* Basit hata yönetimi ve kullanıcı geri bildirimi.

## Kurulum

1.  **Projeyi Klonlayın veya İndirin:**
    Bu dosyaları bilgisayarınıza indirin ve `pd_ai_generator` adında bir klasöre yerleştirin.

2.  **Python Kurulumu:**
    Bilgisayarınızda Python 3.7 veya daha yeni bir sürümün kurulu olduğundan emin olun.

3.  **Sanal Ortam Oluşturun (Önerilir):**
    Proje klasöründeyken bir sanal ortam oluşturun ve etkinleştirin:
    ```bash
    python -m venv venv
    # Windows
    .\venv\Scripts\activate
    # macOS/Linux
    source venv/bin/activate
    ```

4.  **Gerekli Kütüphaneleri Kurun:**
    ```bash
    pip install -r requirements.txt
    ```

5.  **OpenAI API Anahtarını Ayarlayın:**
    * Proje ana dizinindeki `.env.example` dosyasını kopyalayın ve adını `.env` olarak değiştirin.
    * Bir OpenAI API anahtarı edinin: [https://platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys)
    * `.env` dosyasını bir metin düzenleyiciyle açın ve `YOUR_API_KEY_HERE` kısmına kendi API anahtarınızı yapıştırın. Dosyayı kaydedin.
    ```
    OPENAI_API_KEY="sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
    ```
    **ÖNEMLİ:** `.env` dosyasını asla herkese açık bir depoya (GitHub gibi) yüklemeyin! `.gitignore` dosyası bunu engellemeye yardımcı olur.

## Kullanım

1.  Sanal ortamınızın etkin olduğundan emin olun.
2.  Uygulamayı başlatın:
    ```bash
    python main.py
    ```
3.  Açılan pencerede, alt kısımdaki giriş alanına oluşturmak istediğiniz Pd yamasıyla ilgili bir komut yazın (Örn: "440 Hz sinüs dalgası üreten ve ses çıkışına bağlayan bir yama yap").
4.  "Gönder" butonuna tıklayın veya Enter tuşuna basın.
5.  Asistanın yanıtını bekleyin. Başarılı olursa, yamayı kaydetmek için bir dosya diyalogu açılacaktır.
6.  Yamanın kaydedileceği yeri ve dosya adını seçin.
7.  Oluşturulan `.pd` dosyasını Pure Data ile açın.

## Sınırlamalar ve Bilinmesi Gerekenler

* **API Maliyeti:** OpenAI API kullanımı ücretli olabilir. Kullanımınızı takip edin.
* **AI Yetenekleri:** AI'nın üreteceği yamanın kalitesi ve doğruluğu, verdiğiniz komutun netliğine ve AI modelinin (örn: GPT-4o, GPT-3.5-turbo) yeteneklerine bağlıdır. Çok karmaşık veya belirsiz istekler beklenen sonucu vermeyebilir.
* **Müzikalite:** "Uyumlu", "güzel sesli" gibi subjektif isteklerin karşılanması garanti değildir. AI genellikle yapısal olarak doğru ama müzikal olarak basit veya deneysel yamalar üretebilir.
* **Abstraksiyonlar:** Eğer AI, `[pd piyano]` gibi bir alt yama (abstraksiyon) kullanılmasını önerirse, bu `piyano.pd` dosyasının sizin Pure Data arama yolunuzda bulunması veya oluşturulan ana yama ile aynı klasörde olması gerekir. Bu uygulama abstraksiyon dosyalarını otomatik olarak *oluşturmaz*.
* **Nesne Yerleşimi:** Oluşturulan `.pd` dosyasındaki nesnelerin yerleşimi (x, y koordinatları) otomatiktir ve her zaman en okunaklı sonucu vermeyebilir.
* **Hata Yönetimi:** Temel hata yönetimi mevcut olsa da, AI'dan gelen beklenmedik veya hatalı JSON formatları sorunlara yol açabilir.
* **Geliştirme Aşaması:** Bu kod temel bir başlangıç noktasıdır ve daha sağlam hata kontrolü, daha iyi nesne yerleşimi, abstraksiyon yönetimi gibi konularda geliştirilebilir.

## Katkıda Bulunma

Bu projeyi geliştirmek isterseniz, fork yapabilir ve pull request gönderebilirsiniz (Eğer bir GitHub deposu oluşturursanız).
```

**Önemli Notlar:**

* **API Anahtarı Güvenliği:** API anahtarınızı asla kodun içine doğrudan yazmayın veya herkese açık yerlerde paylaşmayın. `.env` dosyası ve `.gitignore` kullanımı bu konuda yardımcı olur.
* **Test ve Geliştirme:** Bu kodlar temel bir başlangıç noktasıdır. Özellikle `openai_handler.py` içindeki `system_message` (prompt) ve `pd_generator.py` içindeki `.pd` formatlama mantığı, daha iyi sonuçlar almak için deneme yanılma ve iyileştirme gerektirebilir.
* **Abstraksiyonlar:** Kullanıcının "piyano sesi" gibi bir şey istemesi durumunda, AI'ın bunu nasıl yorumlayacağı (basit bir `[osc~]` mi, yoksa `[pd piano_synth]` gibi bir abstraksiyon mu önereceği) belirsizdir. Abstraksiyonların yönetimi projenin daha karmaşık bir yönüdür. Başlangıçta, AI'dan standart Pd nesnelerini kullanmasını istemek daha güvenilir olabilir.
* **Hata Durumları:** AI'nın her zaman mükemmel JSON döndürmeyeceğini veya mantıklı bağlantılar kuramayacağını varsayarak kodunuza daha fazla kontrol ve hata yakalama mekanizması eklemek iyi bir fikirdir.

Bu dosyaları alıp belirttiğim klasör yapısına yerleştirerek ve kurulum adımlarını takip ederek projeyi çalıştırabilirsin. Umarım bu başlangıç noktası işine yarar!